# Import

In [1]:
import numpy as np
import pandas as pd
import csv
import datetime
import math

# myLibrary

In [2]:
#製造answer.csv的第1列
def answer_header():
    header = ['user_id']
    for i in range(28):
        header.append('time_slot_' + str(i) )
    return header

In [3]:
def get_week(date_str):
    d = get_date(date_str)
    w = datetime.datetime.strftime(d, '%w')
    return int(w)

In [4]:
def get_date(date_str):
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

In [5]:
def get_slot(time_str):
    the_time = datetime.datetime.strptime(time_str, '%H:%M:%S.%f')
    time_sec0 = datetime.datetime.strptime('01:00:00.00', '%H:%M:%S.%f')
    time_sec1 = datetime.datetime.strptime('09:00:00.00', '%H:%M:%S.%f')
    time_sec2 = datetime.datetime.strptime('17:00:00.00', '%H:%M:%S.%f')
    time_sec3 = datetime.datetime.strptime('21:00:00.00', '%H:%M:%S.%f')
    time_sec4 = datetime.datetime.strptime('23:59:59.99', '%H:%M:%S.%f')
    if time_sec0 <= the_time and the_time <= time_sec1:
        slot = 0
    elif time_sec1 <= the_time and the_time <= time_sec2:
        slot = 1
    elif time_sec2 <= the_time and the_time <= time_sec3:
        slot = 2
    elif time_sec3 <= the_time and the_time <= time_sec4:
        slot = 3
    else:
        slot = 4
    return slot

In [6]:
def get_days(end, start):
    d = end - start
    return d.days

In [7]:
def update_week(week, date_str, event_week, addition):
    date = get_date(date_str)
    end_date = get_date('2017-08-14')
    days = get_days(end_date, date)
    weight = 0
    for i in range(1,33):
        if (days/7) <= i:
            weight = math.pow(0.7, i)
            break
    addition = list(np.array(addition) * weight)
    week.days[event_week].add_slots(addition)
    another_addition = [0, 0, 0]
    another_addition.append(addition[4])
    week.days[event_week-1].add_slots(another_addition)
    
    yesterday_weight_addition = list((np.array(week.days[event_week-1].get_slots()) * 0.3)+(np.array(week.days[event_week-2].get_slots()) * 0.25)+(np.array(week.days[event_week-3].get_slots()) * 0.2)+(np.array(week.days[event_week-4].get_slots()) * 0.15)+(np.array(week.days[event_week-5].get_slots()) * 0.1))
    week.days[event_week].add_slots(yesterday_weight_addition)
    
    return week

In [8]:
def error_list():
    errorlist= ['error', 'api error', 'limit playzone countdown']
    return errorlist

# myClass

In [9]:
class Day():
    def __init__(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def reset(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def get_slots(self):
        return [self.slot0, self.slot1, self.slot2, self.slot3]
    def add_slots(self, addition):
        self.slot0 += addition[0]
        self.slot1 += addition[1]
        self.slot2 += addition[2]
        self.slot3 += addition[3]

In [10]:
class Week():
    def __init__(self):
        self.days = []
        for i in range(7):
            self.days.append(Day())
    def reset(self):
        for i in range(7):
            self.days[i].reset()
    def get_answer(self, id):
        tmp = []
        for i in range(1, 7):
            tmp += self.days[i].get_slots()
        tmp += self.days[0].get_slots()
        answer = [id] + list(np.array(tmp))
        return answer

# Main

In [11]:
with open('answer_v62.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(answer_header())
    week = Week()
    # 開始處理資料
    print(datetime.datetime.now())
    for data_count in range(46, 76): # revise
        # 進度顯示
        print(data_count)
        
        data = pd.read_csv('private/data-0' + str(data_count) + '.csv')
        
        # 取得目前csv檔案的首尾user_id
        head = data.head(1)['user_id'].values[0]
        tail = data.tail(1)['user_id'].values[0]
        
        # 依序處理各id資料並寫入answer.csv
        for id in range(head, tail+1):
            id_data = data[data['user_id'] == id]
            length = len(id_data)
            
            # 依序處理某id的每一筆資料
            now_date = id_data['event_time'].values[0].split()[0]
            addition = [0, 0, 0, 0, 0]
            for i in range(length):
                # 切分時間獲得星期和slot
                event_time = id_data['event_time'].values[i].split()
                date = event_time[0]
                
                # 當日期變化
                if date != now_date:
                    # 將前日的紀錄更新進week
                    event_week = get_week(now_date)
                    week = update_week(week, now_date, event_week, addition)
                    #並初始化now_date和addition
                    now_date = date
                    addition = [0, 0, 0, 0, 0]
                
                # 若無error則把紀錄活動至addition
                trigger = id_data['action_trigger'].values[i]
                if trigger not in error_list():
                    event_slot = get_slot(event_time[1])
                    addition[event_slot] = 1
            
            # 補齊最後一段日期的紀錄
            event_week = get_week(now_date)
            week = update_week(week, now_date, event_week, addition)
            
            # 寫入answer.csv
            writer.writerow(week.get_answer(id))
            
            # 初始化week
            week.reset()
            
            #break # revise
            
    print(datetime.datetime.now())
    
# test

2018-12-05 13:02:45.167371
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
2018-12-05 13:52:52.092183


# Test Area

In [12]:
id_data = data[data['user_id'] == 57159]
id_data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer


In [13]:
data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
0,93913,188172,9206492,16,2017-01-05 14:01:01.248,2,interrupt,iOS,16,17,wifi,False
1,93913,188172,9206492,16,2017-01-05 14:02:06.008,55,leave,iOS,17,17,wifi,False
2,93913,188172,9206492,16,2017-01-05 14:19:23.126,1029,interrupt,iOS,16,17,wifi,False
3,93913,188172,9206492,16,2017-01-05 15:02:57.054,2587,leave,iOS,16,17,wifi,False
4,93913,188172,9206492,16,2017-01-05 15:19:36.894,992,interrupt,iOS,17,17,wifi,False
5,93913,188172,9206492,16,2017-01-05 15:21:17.121,7,seek,iOS,17,17,wifi,False
6,93913,188172,9206492,16,2017-01-05 15:21:21.060,2,seek,iOS,17,17,wifi,False
7,93913,188172,9206492,16,2017-01-05 15:21:25.139,2,seek,iOS,17,17,wifi,False
8,93913,188172,9206492,16,2017-01-05 15:51:49.170,1817,seek,iOS,17,17,wifi,False
9,93913,188172,9206492,16,2017-01-05 15:51:53.570,3,seek,iOS,17,17,wifi,False


# Check Answer

In [14]:
answer = pd.read_csv('answer_v62.csv')
answer

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,57159,1620.633541,1872.059844,1494.604004,2.222485e+03,2117.661410,2.446558e+03,1.953306e+03,2904.350257,1939.855478,...,2.716310e+03,4.037188e+03,1.168167e+03,1.349085e+03,1.077679e+03,1602.696030,3.491043e+03,4.032873e+03,3.221017e+03,4787.853714
1,57160,4.460257,9.239294,5.639776,6.432281e-01,5.379274,1.132059e+01,6.791413e+00,0.749006,8.509990,...,1.956882e+00,2.336767e-01,2.006675e+00,3.745438e+00,2.579396e+00,0.311693,3.883941e+00,7.364639e+00,5.350618e+00,0.579438
2,57161,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.000000e+00,3.430000e-01,0.000000e+00,0.000000e+00,0.490000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
3,57162,0.000000,0.000000,0.000000,0.000000e+00,0.014933,1.022444e+00,2.787962e-01,0.000000,0.000000,...,1.448207e+00,0.000000e+00,3.239991e-02,1.915822e-01,4.153160e-01,0.000000,2.494539e-02,4.374180e-01,5.331369e-01,0.000000
4,57163,0.490000,0.490000,0.000000,0.000000e+00,0.637000,6.370000e-01,0.000000e+00,0.000000,0.313600,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
5,57164,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000e+00,5.002900e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.343000,0.000000e+00,3.430000e-01,0.000000e+00,0.102900
6,57165,4.833771,24.914807,8.680499,1.308776e+01,6.097944,3.639182e+01,1.289947e+01,19.563688,1.875843,...,7.378821e+00,1.163975e+01,3.503780e+00,1.842883e+01,6.745380e+00,9.934854,4.409793e+00,2.441417e+01,8.667895e+00,13.306377
7,57166,0.343144,2.829561,0.331730,0.000000e+00,0.401928,3.442187e+00,3.665690e-01,0.000000,0.466093,...,4.900000e-01,0.000000e+00,1.328538e-01,1.534720e+00,1.470000e-01,0.000000,4.787555e-01,2.071723e+00,6.566000e-01,0.000000
8,57167,562207.594287,696021.940310,249470.797845,1.982598e+04,675826.730251,8.366837e+05,2.998872e+05,23832.692072,803341.810650,...,5.146420e+05,4.089966e+04,1.251466e+06,1.549334e+06,5.553181e+05,44132.319542,1.526248e+06,1.889518e+06,6.772481e+05,53822.362038
9,57168,945007.486551,886616.736216,0.000000,1.130083e+06,466442.682311,4.376217e+05,0.000000e+00,557793.443905,568535.877078,...,0.000000e+00,1.061595e+06,1.596742e+05,1.498078e+05,0.000000e+00,190945.848089,5.943034e+05,5.575817e+05,0.000000e+00,710696.366331
